In [ ]:
# linear algebra
import numpy as np 

# data processing
import pandas as pd 

# data visualization
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style

# Algorithms
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import cross_val_score

import re

In [ ]:
test_df = pd.read_csv("data/titanic_survival/test.csv")
train_df = pd.read_csv("data/titanic_survival/train.csv")
#sns.displot(x=train_df["Fare"],kind="hist")

In [ ]:
#this is testing data's significance
    #removing data without useful patterns or trends


    #adding 2 features.
    #why?
    #wouldnt knowing the sibcount and if married be treated differently?

    #perform data grabbing, only rows with partners & no sibcount, then with partners & siblingcount & siblingcount no partner & no siblingcount and no partner
    #how to only grab rows that pass
    #This data is significant so dont compound to one feature
spouse_df = train_df[(train_df["Parch"] > 0) & (train_df["SibSp"] == 0) ] #23:50 #Not_Survived:Survived
partner_and_non_spouse_relatives_df = train_df[(train_df["Parch"] > 0) & (train_df["SibSp"] > 0) ] #80:58
non_spouse_relatives_df = train_df[(train_df["Parch"] == 0) & (train_df["SibSp"] > 0) ] #70:70
no_relatives = train_df[(train_df["Parch"] == 0) & (train_df["SibSp"] == 0) ] #370:155
plt.hist(x=spouse_df["Survived"]) 

In [ ]:
data = [train_df,test_df]

deck_numeric_dictionary = {
    "A":1,
    "B":2,
    "C":3,
    "D":4,
    "E":5,
    "F":6,
    "G":7,
    "U":8
}

for dataset in data:
    #extracting and converting data
    #havent tested the sinificance of Deck survival rate

    dataset["Deck"] = dataset["Cabin"].fillna("U0")
    dataset["Deck"] = dataset["Deck"].map( lambda x: re.compile("([a-zA-Z]+)").search(x).group())
    dataset["Deck"] = dataset["Deck"].map(deck_numeric_dictionary)
    dataset["Deck"] = dataset["Deck"].fillna(0)
    print(dataset["Deck"].unique())
    dataset["Deck"] = dataset["Deck"].astype(int)
    
    # test_deck = dataset[dataset["Deck"] == 8.0]
    # #8:7
    # #12:35
    # #24:35
    # #8:25
    # #8:24
    # #5:8
    # #2:2
    # #480:210
    # #plt.hist(x=test_deck["Survived"]) 
    # dataset["Deck"]

    
    # dataset["Deck_Type_Sum"] = dataset["Deck"].map(lambda x : type(x))

    # #are the numbers not floats but numpy objects? no they displayed as floats
    
    # print(dataset["Deck_Type_Sum"])
    # dataset["Deck_Type_Sum"] = dataset["Deck"].map(lambda x : type(x) == type(float))
    # print(dataset["Deck_Type_Sum"])
    # print(dataset["Deck_Type_Sum"].sum())
    # dataset["Deck_Type_Sum"].values
    # dataset["Deck_Is_In_0_8_Range"] = ((dataset["Deck"] >= 0) & (dataset["Deck"] < 8))


In [ ]:
dataset["Deck"].value_counts()

In [ ]:
#dropping data (should be at the end
for dataset in data:
    dataset = dataset.drop(columns=["Ticket","PassengerId"])

print(train_df)
print(test_df)

In [ ]:
#formatting data

gendermap = {
    "male":0,
    "female":1
}

embarkedmap = {
    "S":0,
    "C":1,
    "Q":2
}


for dataset in data:
    #dont need to make a group because data meaning is the same for both
    dataset["Sex"] = dataset["Sex"].map(gendermap)
    dataset["Embarked"] = dataset["Embarked"].map(embarkedmap)
    dataset["Embarked"] = dataset["Embarked"].fillna(0)
    dataset["Embarked"] = dataset["Embarked"].astype(int)

print(train_df)

min =0
max =20
age_df = train_df[(train_df["Age"] >= min) & (train_df["Age"] < max)]
plt.hist(x=age_df["Survived"])

In [ ]:
#changing format of age feature
for dataset in data:
    dataset["Age_Group"] = dataset["Age"].fillna(0)
    dataset["Age_Group"] = dataset["Age_Group"].astype(int)
    
    dataset.loc[dataset["Age"] < 11,"Age_Group"] = 0
    dataset.loc[(dataset["Age"] >= 11) & (dataset["Age"] < 18) ,"Age_Group"] = 1
    dataset.loc[(dataset["Age"] >= 18) & (dataset["Age"] < 22) ,"Age_Group"] = 2
    dataset.loc[(dataset["Age"] >= 22) & (dataset["Age"] < 27) ,"Age_Group"] = 3
    dataset.loc[(dataset["Age"] >= 27) & (dataset["Age"] < 33) ,"Age_Group"] = 4
    dataset.loc[(dataset["Age"] >= 33) & (dataset["Age"] < 40) ,"Age_Group"] = 5
    dataset.loc[(dataset["Age"] >= 40) & (dataset["Age"] < 66) ,"Age_Group"] = 6
    dataset.loc[dataset["Age"] >= 66 ,"Age_Group"] = 7

print(train_df)

In [ ]:
#retest the significance of parch nad SibSp parch = Parent or children count

data = [train_df, test_df]
titles = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

for dataset in data:
    # extract titles
                                                    #this only selects a group in the setting of the outside of the group.
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    print(dataset["Title"])
    #print(is " " in d)
    # replace titles with a more common title or as Rare
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    # convert titles into numbers
    dataset['Title'] = dataset['Title'].map(titles)
    # filling NaN with 0, to get safe
    dataset['Title'] = dataset['Title'].fillna(0)
    dataset["Title"] = dataset["Title"].astype(int)

In [ ]:
#making grouped fare feature
for dataset in data:
    dataset["Grouped_Fare"] = dataset["Fare"].fillna(0)
    dataset["Grouped_Fare"] = dataset["Grouped_Fare"].astype(int)
    dataset.loc[dataset["Fare"] < 7.91, "Grouped_Fare"] = 0
    dataset.loc[(dataset["Fare"] >= 7.91) & (dataset["Fare"] < 14.454), "Grouped_Fare"] = 1
    dataset.loc[(dataset["Fare"] >= 14.454) & (dataset["Fare"] < 31), "Grouped_Fare"] = 2
    dataset.loc[(dataset["Fare"] >= 31) & (dataset["Fare"] < 99), "Grouped_Fare"] = 3
    dataset.loc[(dataset["Fare"] >= 99) & (dataset["Fare"] < 250), "Grouped_Fare"] = 4
    dataset.loc[dataset["Fare"] >= 250, "Grouped_Fare"] = 5
    print(dataset)

In [ ]:
for dataset in data:
    #this assumes that one person pays for all relatives and partner.
    #how likely are 2+ people to be old and have siblings aboard?
    new_df = dataset[(dataset["Age"] >= 18) & ((dataset["SibSp"] >= 2) | (dataset["Title"] == 2))]
    print(len(new_df)/len(dataset)*100)
    #comb throught the data, try to understand it
    # for num in range(6):
    #     print("*"*10)
    #     print(dataset["Fare"][(dataset["Deck"] == num) & ((dataset["SibSp"] == 0) & (dataset["Parch"] == 0))])
    # print("*"*30)
    # for num in range(6):
    #     print("*"*10)
    #     print(dataset["Fare"][(dataset["Deck"] == num) & ((dataset["SibSp"] != 0) | (dataset["Parch"] != 0))])

#test relatives
for dataset in data:
    dataset['Fare_Per_Person'] = dataset['Fare']/((dataset['SibSp']) + (dataset['Parch']))
    dataset.replace([np.inf, -np.inf], 0, inplace=True)
    dataset["Fare_Per_Person"] = dataset["Fare_Per_Person"].fillna(0)
    print(dataset["Fare_Per_Person"])

    dataset['Fare_Per_Person'] = dataset['Fare_Per_Person'].astype(int)
    print(dataset["Fare_Per_Person"])


    #print(dataset["Fare"][(dataset["Deck"] == 1) & (dataset["Parch"] > 0)])
    #deck 2 has significantly higher 
    #does fare cost co inside with deck?
        #yes but the majority of the values stay in a set range.
    #how does machine learning handle larger values?

#     dataset['Fare_Per_Person'] = dataset['Fare']/((dataset['relatives']+1)+(dataset['Parch']))
#     dataset['Fare_Per_Person'] = dataset['Fare_Per_Person'].astype(int)# Let's take a last look at the training set, before we start training the models.
# train_df.head(10)

In [ ]:
train_df = train_df.drop(columns = ["Name", "PassengerId","Ticket","Age","Fare","Cabin","Parch"])
test_df = test_df.drop(columns = ["Name", "PassengerId","Ticket","Age","Fare","Cabin","Parch"])
#count is alvive generalizes data, is that something you want?

print(train_df)
print(test_df)

In [ ]:
x_train = train_df.drop("Survived",axis=1)
y_train = train_df["Survived"]

x_test = test_df

# #no connection to the data. its simply grouped
# no parch but sibsp = good
# sibsp but no parch = good
# output of no parch but sibsp and output of sibsp and no parch = same. except more specific
# simplied value, no need for ranges or many leaves.
# when has a range of numbers many leaves form

In [ ]:
#find out whats benefictual n_estimators based on the data.
rf = RandomForestClassifier(n_estimators=100)
rf.fit(x_train, y_train)



#what does predict do?
#change attributes of the RandomForestClassifier
#the perdiction does effect the .score function
#sometimes code doesnt make sense untl you look at later code.
rf_pred = rf.predict(x_test)

rf.score(x_train, y_train)
acc_random_forest = round(rf.score(x_train, y_train) * 100, 2)

In [ ]:
sgd = linear_model.SGDClassifier(max_iter=5, tol=None)
sgd.fit(x_train, y_train)
sgd_pred = sgd.predict(x_test)

sgd.score(x_train, y_train)

acc_sgd = round(sgd.score(x_train, y_train) * 100, 2)

In [ ]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)

logreg_pred = logreg.predict(x_test)

acc_log = round(logreg.score(x_train, y_train) * 100, 2)

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(x_train, y_train)

knn_pred = knn.predict(x_test)
acc_knn = round(knn.score(x_train, y_train) * 100, 2)

In [ ]:
gaussian = GaussianNB()
gaussian.fit(x_train, y_train)
gaus_pred = gaussian.predict(x_test)

acc_gaussian = round(gaussian.score(x_train, y_train) * 100, 2)

In [ ]:
# import warnings
# with warnings.catch_warnings():
#  # ignore all caught warnings
#  warnings.filterwarnings("ignore")
#  # execute code that will generate warnings

perceptron = Perceptron(max_iter=5)
perceptron.fit(x_train, y_train)

perceptron_pred = perceptron.predict(x_test)

acc_perceptron = round(perceptron.score(x_train, y_train) * 100, 2)

In [ ]:
linear_svc = LinearSVC()
linear_svc.fit(x_train, y_train)

l_svc_pred = linear_svc.predict(x_test)

acc_linear_svc = round(linear_svc.score(x_train, y_train) * 100, 2)

In [ ]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(x_train, y_train)
dt_pred = decision_tree.predict(x_test)

acc_decision_tree = round(decision_tree.score(x_train, y_train) * 100, 2)

In [ ]:
results = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 
              'Decision Tree'],
    'Score': [acc_linear_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_decision_tree]})
result_df = results.sort_values(by='Score', ascending=False)
result_df = result_df.set_index('Score')
result_df.head(9)

In [ ]:
scores = cross_val_score(rf, x_train, y_train, cv=10, scoring = "accuracy")
print("Scores:", scores)
print("Mean:", scores.mean())
print("Standard Deviation:", scores.std())

In [ ]:
importantances = pd.DataFrame({"features":x_train.columns,"importantance":np.round(rf.feature_importances_,3)})
importantances = importantances.sort_values(by='importantance', ascending=False)
importantances.head(10)